# PySpark Common Operations

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.getOrCreate()

### Use case: Data Profiling 

In [17]:
from datetime import datetime, date
import pandas as pd
from pyspark.sql import Row

df = spark.createDataFrame([
    Row(a=1, b=2., c='string1', d=date(2000, 1, 1), e=datetime(2000, 1, 1, 12, 0)),
    Row(a=2, b=3., c='string2', d=date(2000, 2, 1), e=datetime(2000, 1, 2, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)),
    Row(a=4, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)), # Totally duplicate
    Row(a=4, b=5., c='string3', d=date(2000, 5, 1), e=datetime(2000, 1, 3, 12, 0)), # Part duplicate
    Row(a=None, b=5., c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)), # PK contains Null
    Row(a=6, b=None, c='string3', d=date(2000, 3, 1), e=datetime(2000, 1, 3, 12, 0)) # PK contains Null


])
df

DataFrame[a: bigint, b: double, c: string, d: date, e: timestamp]

In [4]:
df.show()

+----+---+-------+----------+-------------------+
|   a|  b|      c|         d|                  e|
+----+---+-------+----------+-------------------+
|   1|2.0|string1|2000-01-01|2000-01-01 12:00:00|
|   2|3.0|string2|2000-02-01|2000-01-02 12:00:00|
|   4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
|   4|5.0|string3|2000-03-01|2000-01-03 12:00:00|
|   4|5.0|string3|2000-05-01|2000-01-03 12:00:00|
|null|5.0|string3|2000-03-01|2000-01-03 12:00:00|
+----+---+-------+----------+-------------------+



In [18]:
PK = ['a','b','c']


In [23]:
# PK null check
from pyspark.sql.functions import lit
null_check_str = ' is null or '.join(PK) + ' is null'
df_null = df.filter(null_check_str)
df_null = df_null.withColumn('desc',lit('PK Contains Null'))
df_null.show()

+----+----+-------+----------+-------------------+----------------+
|   a|   b|      c|         d|                  e|            desc|
+----+----+-------+----------+-------------------+----------------+
|null| 5.0|string3|2000-03-01|2000-01-03 12:00:00|PK Contains Null|
|   6|null|string3|2000-03-01|2000-01-03 12:00:00|PK Contains Null|
+----+----+-------+----------+-------------------+----------------+



In [25]:
from pyspark.sql.functions import count

In [ ]:
df = 